## google drive mount

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
%ls

drive/  sample_data/


## clone yolov5

In [ ]:
%cd /content
!git clone https://github.com/ultralytics/yolov5.git

/content
Cloning into 'yolov5'...
remote: Enumerating objects: 12281, done.
remote: Total 12281 (delta 0), reused 0 (delta 0), pack-reused 12281
Receiving objects: 100% (12281/12281), 12.02 MiB | 25.28 MiB/s, done.
Resolving deltas: 100% (8475/8475), done.


In [ ]:
%cd /content/yolov5/
!pip install -r requirements.txt

/content/yolov5
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 596 kB 5.2 MB/s 
  Attempting uninstall: PyYAML
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
%cd ..

/content


In [ ]:
%ls

drive/  sample_data/  yolov5/


In [ ]:
%cd drive/MyDrive/food-detection/data/

/content/drive/MyDrive/food-detection/data


In [ ]:
%ls

category.txt    images/  multiple_food.txt  train.txt
food_data.yaml  labels/  train.cache        val.txt


In [ ]:
%cd /
from glob import glob

img_list = glob('/content/drive/MyDrive/food-detection/data/images/*/*.jpg')

print(len(img_list))

/
14365


## split data set

In [ ]:
from sklearn.model_selection import train_test_split

train_img_list, val_img_list = train_test_split(img_list, test_size=0.2, random_state=2000)

print(len(train_img_list), len(val_img_list))

11492 2873


In [ ]:
with open('/content/drive/MyDrive/food-detection/data/train.txt', 'w') as f:
  f.write('\n'.join(train_img_list) + '\n')

with open('/content/drive/MyDrive/food-detection/data/val.txt', 'w') as f:
  f.write('\n'.join(val_img_list) + '\n')

## labeling

In [ ]:
import os
from PIL import Image

In [ ]:
# Read '/content/drive/MyDrive/food-detection/data/images/*/bb_info.txt'

for i in range(100): # folder 1~100
  with open(f'/content/drive/MyDrive/food-detection/data/images/{i+1}/bb_info.txt', 'r') as f: # read each bb_info.txt files in 1~100 folders, {i+1} is object_class_id

    # Read txt files line by line and split them
    lines = f.readlines()
    for j in range(1, len(lines)):
      split_line = lines[j].split()

      # if not exist dir: mkdir
      if not os.path.exists(f'/content/drive/MyDrive/food-detection/data/labels/{i+1}'):
        os.mkdir(f'/content/drive/MyDrive/food-detection/data/labels/{i+1}')
      
      # index 0 -> label txt file name
      with open(f'/content/drive/MyDrive/food-detection/data/labels/{i+1}/{split_line[0]}.txt', 'w') as file: # folder index...

        img = Image.open(f'/content/drive/MyDrive/food-detection/data/images/{i+1}/{split_line[0]}.jpg')
        dw = 1./img.size[0]
        dh = 1./img.size[1]

        # transfer under line to a lower line
        # [image-number] [top-left-X] [top-left-Y] [bottom-right-X] [bottom-right-Y]
        # [object-class-id] [center-X] [center-Y] [width] [height]

        # [center-X] = ([top-left-X] + [bottom-right-X] / 2) / [img_width]
        centerX = (int(split_line[1]) + int(split_line[3]) / 2.0) * dw
        # [center-Y] = ([top-left-Y] + [bottom-right-Y] / 2) / [img_height]
        centerY = (int(split_line[2]) + int(split_line[4]) / 2.0) * dh
        # [width] = ([bottom-right-X] - [top-left-X]) / [img_width]
        width = abs(int(split_line[3]) - int(split_line[1])) * dw
        # [height] = ([top-left-Y] - [bottom-right-Y]) / [img_height]
        height = abs(int(split_line[4]) - int(split_line[2])) * dh

        label = ' '.join([str(i), str(centerX), str(centerY), str(width), str(height)])
        file.write(label + '\n')

## yaml

In [ ]:
import yaml

with open('/content/drive/MyDrive/food-detection/data/food_data.yaml', 'r') as f:
  data = yaml.safe_load(f)

print(data)

# data['train'] = '/content/drive/MyDrive/food-detection/data/train.txt'
# data['val'] = '/content/drive/MyDrive/food-detection/data/val.txt'

# Read txt files line by line and split them
with open('/content/drive/MyDrive/food-detection/data/category.txt', 'r') as f:
  lines = f.readlines()
  for i in range(1, len(lines)):
    split_line = lines[i].split('\t')
    data['names'].append(split_line[1].strip('\n'))

with open('/content/drive/MyDrive/food-detection/data/food_data.yaml', 'w') as f:
  yaml.dump(data, f)

print(data)

{'train': '/content/drive/MyDrive/food-detection/data/train.txt', 'val': '/content/drive/MyDrive/food-detection/data/val.txt', 'nc': 100, 'names': []}
{'train': '/content/drive/MyDrive/food-detection/data/train.txt', 'val': '/content/drive/MyDrive/food-detection/data/val.txt', 'nc': 100, 'names': ['rice', 'eels on rice', 'pilaf', "chicken-'n'-egg on rice", 'pork cutlet on rice', 'beef curry', 'sushi', 'chicken rice', 'fried rice', 'tempura bowl', 'bibimbap', 'toast', 'croissant', 'roll bread', 'raisin bread', 'chip butty', 'hamburger', 'pizza', 'sandwiches', 'udon noodle', 'tempura udon', 'soba noodle', 'ramen noodle', 'beef noodle', 'tensin noodle', 'fried noodle', 'spaghetti', 'Japanese-style pancake', 'takoyaki', 'gratin', 'sauteed vegetables', 'croquette', 'grilled eggplant', 'sauteed spinach', 'vegetable tempura', 'miso soup', 'potage', 'sausage', 'oden', 'omelet', 'ganmodoki', 'jiaozi', 'stew', 'teriyaki grilled fish', 'fried fish', 'grilled salmon', 'salmon meuniere', 'sashimi',

## resize img
필요없을듯

In [ ]:
# import cv2
# import imutils

In [ ]:
# with open('/content/drive/MyDrive/food-detection/data/images/*/*.jpg', 'rw') as f:
#   # load img
#   img = cv2.imread(f, cv2.IMREAD_COLOR)
#   # resize img
#   resized_img = imutils.resize(img, width=200, inter=cv2.INTER_AREA)
#   # save img 
#   cv2.imwrite(f, resized_img)

## gogo yolo!

In [ ]:
%cd /content/yolov5/

/content/yolov5


In [ ]:
!python train.py --img 320 --rect --batch 32 --epochs 30 --data /content/drive/MyDrive/food-detection/data/food_data.yaml --cfg ./models/yolov5m.yaml --weights yolov5m.pt --name food_yolov5m_result

train: weights=yolov5m.pt, cfg=./models/yolov5m.yaml, data=/content/drive/MyDrive/food-detection/data/food_data.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=30, batch_size=32, imgsz=320, rect=True, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train, name=food_yolov5m_result, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 v6.1-261-g19f33cb Python-3.7.13 torch-1.11.0+cu113 CPU

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0, obj_pw=1.0, iou_t=0.2, anchor_t=4.0,

In [ ]:
%load_ext tensorboard
%tensorboard --logdir /content/yolov5/runs/

In [ ]:
from IPython.display import Image
import os

val_img_path = val_img_list[0]

!python detect.py --weights /content/yolov5/runs/train/food_yolov5m_result/weights/best.pt --img 200 --rect --conf 0.5 --source "{val_img_path}"